#5.13 다층 퍼셉트론으로 손글씨 숫자 분류

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint

##MNIST 데이터 불러오기

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


##데이터 정규화

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

##모델 생성

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)), # 데이터 차원 변경
    Dense(256, activation='relu'), # 첫번째 히든 레이어 (h1)
    Dense(128, activation='relu'), # 두번째 히든 레이어 (h2)
    Dropout(0.1), # 두번째 히든 레이어(h2)에 드랍아웃(10%) 적용
    Dense(10), # 세번째 히든 레이어 (logit)
    Activation('softmax') # softmax layer
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 256)               200960    
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                                 
Total params: 235,146
Trainable params: 235,146
Non-tr

In [ ]:
"""
sparse_categorical_crossentropy:
레이블을 원 핫 인코딩으로 자동으로 변경하여 크로스 엔트로피 측정합니다.
"""
# 손실 함수 및 최적화 정의
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

##조기 종료 (Early Stopping)
매 주기(Epoch)마다 검증 데이터로 검증 정확도를 측정합니다.

검증 정확도가 5번 연속으로 개선되지 않을 시, 조기 종료를 수행합니다.

In [ ]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

In [ ]:
model.fit(x_train, y_train, epochs=300, batch_size=1000, validation_split = 0.1, callbacks=callbacks)

Epoch 1/300
54/54 [==============================] - 2s 28ms/step - loss: 0.6945 - accuracy: 0.8081 - val_loss: 0.2143 - val_accuracy: 0.9413
Epoch 2/300
54/54 [==============================] - 1s 25ms/step - loss: 0.2435 - accuracy: 0.9302 - val_loss: 0.1545 - val_accuracy: 0.9587
Epoch 3/300
54/54 [==============================] - 1s 25ms/step - loss: 0.1766 - accuracy: 0.9484 - val_loss: 0.1177 - val_accuracy: 0.9690
Epoch 4/300
54/54 [==============================] - 1s 26ms/step - loss: 0.1372 - accuracy: 0.9604 - val_loss: 0.1016 - val_accuracy: 0.9710
Epoch 5/300
54/54 [==============================] - 1s 25ms/step - loss: 0.1112 - accuracy: 0.9676 - val_loss: 0.0915 - val_accuracy: 0.9743
Epoch 6/300
54/54 [==============================] - 1s 24ms/step - loss: 0.0913 - accuracy: 0.9734 - val_loss: 0.0856 - val_accuracy: 0.9763
Epoch 7/300
54/54 [==============================] - 1s 25ms/step - loss: 0.0768 - accuracy: 0.9779 - val_loss: 0.0773 - val_accuracy: 0.9777
Epoch 

##학습 정확도

In [ ]:
results = model.evaluate(x_test,  y_test, verbose = 0)
print('test loss, test acc:', results)

test loss, test acc: [0.06988591700792313, 0.9799000024795532]


##모델 테스트

In [ ]:
print(model.predict(x_test))

[[3.8561474e-08 1.3449615e-07 3.8905057e-07 ... 9.9999273e-01
  7.1056849e-09 2.4606103e-07]
 [6.9388367e-10 1.8482262e-07 9.9999976e-01 ... 2.9982909e-09
  1.9697657e-09 4.7155446e-16]
 [2.5481318e-06 9.9932384e-01 2.0088356e-04 ... 1.8431227e-04
  2.1126833e-04 1.3445716e-06]
 ...
 [1.6659596e-12 8.1789790e-11 2.7405031e-11 ... 9.7147426e-07
  1.6597900e-09 1.0115283e-07]
 [1.8852175e-14 1.1116867e-12 7.0766120e-14 ... 5.9200008e-13
  4.4729347e-08 5.1638147e-12]
 [2.0366977e-09 1.8357821e-13 4.7865362e-12 ... 5.3912241e-15
  2.6657946e-13 1.9635755e-14]]


In [ ]:
result_y = model.predict(x_test)
# print(result_y.shape)
print(result_y[0])

[3.8561474e-08 1.3449615e-07 3.8905057e-07 6.3858661e-06 5.7972543e-10
 2.1155229e-08 1.5538711e-10 9.9999273e-01 7.1056849e-09 2.4606103e-07]
